## 1. KoBert 모델 불러오기

In [ ]:
# SKTBrain/KoBert 모델 가져오기. https://github.com/SKTBrain/KoBERT/tree/master/kobert_hf 허깅페이스 업로드 버전
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-azbd9uvw/kobert-tokenizer_ae8afd51f6c344a2b55d7cb92e3ea014
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-azbd9uvw/kobert-tokenizer_ae8afd51f6c344a2b55d7cb92e3ea014
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4633 sha256=7a28e80f5eb2e677081e6e2059ffc39578c0c91cfb9cac8562bd4f9bdf026d4f
  Stored in directory: /tmp/pip-ephem-wheel-cache-t0qqhqhq/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


## 2. Colab 환경 설정

In [ ]:
# Colab 환경 설정
!pip install gluonnlp pandas tqdm
!pip install mxnet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=661754 sha256=97ef41ab2c4eed916dfe04d5ccd901f542f2674751653489651e25c531305bbe
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp


In [ ]:
# 라이브러리 import
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

In [ ]:
# https://stackoverflow.com/questions/74893742/how-to-solve-attributeerror-module-numpy-has-no-attribute-bool # bool 연산자 오류? 인 듯한데, 옆의 링크 참고해서 코드 수정해봤습니다.
import numpy as np
np.bool = np.bool_

In [ ]:
# GPU 설정 (Colab T4 GPU)
device = torch.device("cuda:0")

## 3. 모델 설정

*  transformers :  BERTModel 의 pretrained_model 중 'skt/kobert-base-v1' 사용.
*  tokenizer : KoBERTTokenizer 의 pretrain_model 중 'skt/kobert-base-v1' 사용.





In [ ]:
# transformers 설정
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
from transformers import BertModel
model = BertModel.from_pretrained('skt/kobert-base-v1')

config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [ ]:
# tokenizer 설정
from kobert_tokenizer import KoBERTTokenizer
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
# tokenizer vocab 생성.
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token = '[PAD]')

## 4. 데이터 불러오기

*   AI Hub 감정분류데이터셋 사용.


In [ ]:
data = pd.read_csv("/content/감정분류데이터셋.csv", encoding = 'cp949')

In [ ]:
data.head()

data.shape
# (19374, 15)

data['상황'].unique()
# array(['happiness', 'neutral', 'sadness', 'angry', 'surprise', 'disgust',
#      'fear'], dtype=object)

,wav_id,발화문,상황,1번 감정,1번 감정세기,2번 감정,2번 감정세기,3번 감정,3번 감정세기,4번 감정,4번감정세기,5번 감정,5번 감정세기,나이,성별
0,5f4141e29dd513131eacee2f,헐! 나 이벤트에 당첨 됐어.,happiness,angry,2,surprise,2,happiness,2,happiness,2,happiness,2,48,female
1,5f4141f59dd513131eacee30,내가 좋아하는 인플루언서가 이벤트를 하더라고. 그래서 그냥 신청 한번 해봤지.,happiness,neutral,0,happiness,2,happiness,2,happiness,2,happiness,2,48,female
2,5f4142119dd513131eacee31,"한 명 뽑는 거였는데, 그게 바로 내가 된 거야.",happiness,angry,2,happiness,2,happiness,2,happiness,2,happiness,2,48,female
3,5f4142279dd513131eacee32,"당연히 마음에 드는 선물이니깐, 이벤트에 내가 신청 한번 해본 거지. 비싼 거야. ...",happiness,angry,2,happiness,2,happiness,2,happiness,2,happiness,1,48,female
4,5f3c9ed98a3c1005aa97c4bd,에피타이저 정말 좋아해. 그 것도 괜찮은 생각인 것 같애.,neutral,happiness,2,happiness,1,happiness,2,happiness,1,happiness,1,48,female


#### 4-1. 데이터 전처리

In [ ]:
# 감정 데이터셋 내부 감정 label / '두려움', '놀람', '화남', '슬픔', '중립', '기쁨', '혐오' : 7개 감정 => 각각 0, 1, 2, 3, 4, 5, 6 에 대응
data.loc[(data['상황'] == 'fear'), '상황'] = 0
data.loc[(data['상황'] == 'surprise'), '상황'] = 1
data.loc[(data['상황'] == 'angry'), '상황'] = 2
data.loc[(data['상황'] == 'sadness'), '상황'] = 3
data.loc[(data['상황'] == 'neutral'), '상황'] = 4
data.loc[(data['상황'] == 'happiness'), '상황'] = 5
data.loc[(data['상황'] == 'disgust'), '상황'] = 6

In [ ]:
data['상황'].unique()
# array([5, 4, 3, 2, 1, 6, 0], dtype=object)

array([5, 4, 3, 2, 1, 6, 0], dtype=object)

In [ ]:
data_list = []
for talk, label in zip (data['발화문'], data['상황']):
  temp = []
  temp.append(talk)
  temp.append(str(label))

  data_list.append(data)

In [ ]:
print(data)
# ['걱정해줘서 고마워.', '6']

print(data_list[:4])
# ['헐! 나 이벤트에 당첨 됐어.', '5'], ['내가 좋아하는 인플루언서가 이벤트를 하더라고. 그래서 그냥 신청 한번 해봤지.', '5'], ['한 명 뽑는 거였는데, 그게 바로 내가 된 거야.', '5'], ['당연히 마음에 드는 선물이니깐, 이벤트에 내가 신청 한번 해본 거지. 비싼 거야. 그래서 못 산 향수야.', '5']]

['걱정해줘서 고마워.', '6']
[['헐! 나 이벤트에 당첨 됐어.', '5'], ['내가 좋아하는 인플루언서가 이벤트를 하더라고. 그래서 그냥 신청 한번 해봤지.', '5'], ['한 명 뽑는 거였는데, 그게 바로 내가 된 거야.', '5'], ['당연히 마음에 드는 선물이니깐, 이벤트에 내가 신청 한번 해본 거지. 비싼 거야. 그래서 못 산 향수야.', '5'], ['에피타이저 정말 좋아해. 그 것도 괜찮은 생각인 것 같애.', '4'], ['난 부페 형식의 음식들도 정말 좋아해. 그 것도 좀 알려 줘.', '4'], ['응. 완전히 끝난 거야. 한 달 동안 주말에 쉬지도 못하고 일만 했거든.', '5'], ['신나는 음악 듣는 것도 좋고, 어디 여행 가고 싶고 이 것 저 것 다 해보고 싶어.', '5'], ['친구들도 내 연락 기다리고 있을 텐데 내가 까먹고 있었네?', '5'], ['그래. 일단은 친구들부터 만나서 여행 계획에 대해서 얘기 좀 해봐야 되겠어.', '5']]


#### 4-2. 데이터 분류 (학습, 테스트셋)

In [ ]:
# train 데이터셋 / test 데이터셋 분류
from sklearn.model_selection import train_test_split
dataset_train, dataset_test = train_test_split(data_list, test_size = 0.2, shuffle = True, random_state = 32)

In [ ]:
print(len(dataset_train), len(dataset_test))
# 15499 3875

15499 3875


#### 4-3. 데이터셋 Tokenize

In [ ]:
tok = tokenizer.tokenize

In [ ]:
# tokenization과 encoding, padding 작업 수행.

class BERTDataset(Dataset):
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):

    transform = nlp.data.BERTSentenceTransform(
        bert_tokenizer, max_seq_length = max_len, vocab = vocab, pad = pad, pair= pair)

    self.sentences = [transform([i[sent_idx]]) for i in dataset]
    self.labels = [np.int32(i[label_idx]) for i in dataset]

  def __getitem__(self, i):
    return (self.sentences[i] + (self.labels[i],))

  def __len__(self):
    return (len(self.labels))

#### 4-4. 파라미터 세팅

In [ ]:
# parameter 설정. https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb 참고.
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5  # 1. gpu 가 없을 때 CPU 환경에서 epochs = 3으로 설정. / # 두 번째 테스트에서 T4 GPU 사용. epochs = 5 설정.
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

#### 4-5. 데이터 인코딩, 패딩 작업



In [ ]:
# BERTDataset 함수에 train 데이터셋, test 데이터셋 입력. / 초기 입력값 : sent_idx = 0, label_idx = 1, bert_tokenizer = tok, vocab = vocab, max_len = max_len, pad = True, pair = False
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

In [ ]:
# 데이터 로더 생성.
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


## 5. Bert 분류 모델 불러오기
*  https://github.com/SKTBrain/KoBERT/blob/master/scripts/NSMC/naver_review_classifications_pytorch_kobert.ipynb
참고



In [ ]:
class BERTClassifier(nn.Module):
  def __init__(self, bert, hidden_size = 768, num_classes = 7,
               dr_rate = None, params = None):
    super(BERTClassifier, self).__init__()
    self.bert = bert
    self.dr_rate = dr_rate

    self.classifier = nn.Linear(hidden_size, num_classes)
    if dr_rate:
      self.dropout = nn.Dropout(p = dr_rate)

  def gen_attention_mask(self, token_ids, valid_length):
    attention_mask = torch.zeros_like(token_ids)
    for i, v in enumerate(valid_length):
      attention_mask[i][:v] = 1
    return attention_mask.float()

  def forward(self, token_ids, valid_length, segment_ids):
    attention_mask = self.gen_attention_mask(token_ids, valid_length)

    _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(),
                          attention_mask = attention_mask.float().to(token_ids.device), return_dict = False)
    if self.dr_rate:
      out = self.dropout(pooler)
    return self.classifier(out)

#### 5-1. Bert 모델 불러오기

In [ ]:
bert_model = BERTClassifier(model, dr_rate = 0.5).to(device)

In [ ]:
# optimizer , scheduler 설정.
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params' : [p for n, p in bert_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay' : 0.01},
    {'params' : [p for n, p in bert_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:521: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# 정확도 측정하는 calc_accuracy 함수 정의.
def calc_accuracy(X, Y):
  max_vals, max_indices = torch.max(X, 1)
  train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
  return train_acc
train_dataloader

## 6. 모델 학습

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    bert_model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = bert_model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    bert_model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = bert_model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

<ipython-input-39-195589d725fd>:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/243 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.04405133053660393 train acc 0.984375
epoch 1 batch id 201 loss 0.12313532084226608 train acc 0.9668843283582089
epoch 1 train acc 0.9655349794238683


<ipython-input-39-195589d725fd>:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/61 [00:00<?, ?it/s]

...

epoch 4 test acc 0.9214139344262294


  0%|          | 0/243 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.00755750248208642 train acc 1.0
epoch 5 batch id 201 loss 0.06557511538267136 train acc 0.9789334577114428
epoch 5 train acc 0.9780092592592593


  0%|          | 0/61 [00:00<?, ?it/s]

epoch 5 test acc 0.9251244145199063


## 7. 최종 Validation

In [ ]:
def predict(predict_sentence): # input = 감정분류하고자 하는 sentence

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False) # 토큰화한 문장
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size = batch_size, num_workers = 5) # torch 형식 변환

    bert_model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length = valid_length
        label = label.long().to(device)

        out = bert_model(token_ids, valid_length, segment_ids)


        test_eval = []
        for i in out: # out = model(token_ids, valid_length, segment_ids)
            logits = i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("공포가")
            elif np.argmax(logits) == 1:
                test_eval.append("놀람이")
            elif np.argmax(logits) == 2:
                test_eval.append("분노가")
            elif np.argmax(logits) == 3:
                test_eval.append("슬픔이")
            elif np.argmax(logits) == 4:
                test_eval.append("중립이")
            elif np.argmax(logits) == 5:
                test_eval.append("행복이")
            elif np.argmax(logits) == 6:
                test_eval.append("혐오가")

        print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

## BERT 감정 분류 모델 시현

In [ ]:
# 질문에 0 입력 시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == "0" :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 두 번째 테스트 시작해볼게!
>> 입력하신 내용에서 행복이 느껴집니다.


하고싶은 말을 입력해주세요 : 어무이.. 불효자는 갑니다
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 죽는 날까지 하늘을 우러러 한 점 부끄럼이 없기를,
>> 입력하신 내용에서 행복이 느껴집니다.


하고싶은 말을 입력해주세요 : 잎새에 이는 바람에도 나는 괴로워했다.
>> 입력하신 내용에서 중립이 느껴집니다.


하고싶은 말을 입력해주세요 : 별을 노래하는 마음으로 모든 죽어 가는 것을 사랑해야지
>> 입력하신 내용에서 행복이 느껴집니다.


하고싶은 말을 입력해주세요 : 그리고 나한테 주어진 길을 걸어가야겠다.
>> 입력하신 내용에서 행복이 느껴집니다.


하고싶은 말을 입력해주세요 : 아 ㅋㅋㅋ 성능이 왜그래
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 분노가 아니야
>> 입력하신 내용에서 혐오가 느껴집니다.


하고싶은 말을 입력해주세요 : 혐오도 아니야
>> 입력하신 내용에서 혐오가 느껴집니다.


하고싶은 말을 입력해주세요 : 순전히 너 마음대로구나
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 이제 지친다
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 적당히 해 좀
>> 입력하신 내용에서 혐오가 느껴집니다.


하고싶은 말을 입력해주세요 : 왜 그러는거야?
>> 입력하신 내용에서 분노가 느껴집니다.


하고싶은 말을 입력해주세요 : 여자친구가 "나 이제 지쳤어.." 라고 말하네
>> 입력하신 내용에서 슬픔이 느껴집니다.


하고싶은 말을 입력해주세요 : 남자친구가 "우리 오늘 어디갈까??" 라면서 기대하고 있어
>> 입력하신 내용에서 행복이 느껴집니다.


하고싶은 말을 입력해주세요 : 내가 10주년 기념일에 맞춰서 반지를 새로 맞추자고 했는데, 아내가 "또 맞추려고? 그래, 니 마음대로 해라" 라고 했어. 
>> 입력하